In [ ]:
from IPython import get_ipython
from IPython.display import display

In [ ]:
# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalação e atualização do ffmpeg
!apt-get update -y
!apt-get install -y ffmpeg

In [ ]:
import os
import requests
import multiprocessing
from datetime import datetime, timedelta
import json
import time  # Importando a biblioteca time para adicionar um delay
import subprocess
import math
import re

In [ ]:
# Configuração de diretórios e constantes
# NOTA: OUTPUT_FOLDER temporário no Colab local para FFmpeg, não no Drive.
# A gravação final no Drive será o arquivo JSON.
TEMP_OUTPUT_FOLDER = "/content/temp_recordings"
os.makedirs(TEMP_OUTPUT_FOLDER, exist_ok=True) # Cria o diretório temporário local

BASE_DRIVE_FOLDER = "/content/drive/MyDrive/XCam.Drive"
os.makedirs(BASE_DRIVE_FOLDER, exist_ok=True) # Cria o diretório base no Drive

ABYSS_UPLOAD_URL = 'http://up.hydrax.net/0128263f78f0b426d617bb61c2a8ff43'
RECORD_SECONDS = 180  # 3 minutos para testes (ajuste se necessário)

In [ ]:
# Funções utilitárias
def format_seconds(seconds):
    # Formata segundos para XhYmZs (ignora horas se zero)
    total_seconds = int(seconds)
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60

    parts = []
    if hours > 0:
        parts.append(f"{hours}h")
    if minutes > 0 or (hours == 0 and seconds > 0):
        parts.append(f"{minutes}m")
    if seconds > 0 or total_seconds == 0:
        parts.append(f"{seconds}s")
    return "".join(parts) if parts else "0s"

def log_progress(username, elapsed_seconds, total_seconds=180):
    percent = min((elapsed_seconds / total_seconds) * 100, 100)
    tempo = format_seconds(elapsed_seconds)
    minutos_gravados = math.floor(elapsed_seconds / 60)
    minutos_restantes = max(0, math.ceil((total_seconds - elapsed_seconds) / 60))
    print(f"⏱️ [{username}] Gravados: {minutos_gravados} min | Restantes: {minutos_restantes} min | Tempo total: {tempo} — 📊 {percent:.1f}% concluído")

In [ ]:
# Etapa 1 – Buscar broadcasts da API XCam (com fallback para liveInfo)
def get_broadcasts(limit=5, page=1):
    # Primeira chamada: API principal
    api_url_main = f"https://api.xcam.gay/?limit={limit}&page={page}"
    print(f"🌐 Acessando API principal: {api_url_main}")

    streams_from_main = []
    streams_without_preview = []

    try:
        response_main = requests.get(api_url_main)
        response_main.raise_for_status()
        data_main = response_main.json()

        broadcasts_data = data_main.get("broadcasts")
        if not broadcasts_data:
            print("⚠️ Chave 'broadcasts' não encontrada na resposta da API principal.")
            return []

        items = broadcasts_data.get("items")
        if not isinstance(items, list):
            print(f"⚠️ Chave 'items' não encontrada ou não é uma lista em 'broadcasts' da API principal. Tipo encontrado: {type(items)}")
            return []

        for item in items:
            preview = item.get("preview") or {}
            src = preview.get("src")
            username = item.get("username", "desconhecido")
            if src:
                streams_from_main.append({"username": username, "src": src})
            else:
                print(f"⚠️ Stream sem URL de preview encontrado para usuário (API principal): {username}. Tentando liveInfo...")
                streams_without_preview.append({"username": username})

        print(f"✅ Encontrados {len(streams_from_main)} streams com URL de preview na API principal na página {page}.")

    except requests.exceptions.RequestException as e:
        print(f"❌ Erro na requisição HTTP para a API principal: {e}")
        return []
    except json.JSONDecodeError:
        print(f"❌ Erro ao decodificar resposta JSON da API principal. Resposta: {response_main.text}")
        return []
    except Exception as e:
        print(f"❌ Erro inesperado ao acessar API principal: {e}")
        return []

    # Segunda Chamada (Fallback): API liveInfo para streams sem preview
    streams_from_liveinfo = []
    if streams_without_preview:
        print(f"\nAttempting to fetch liveInfo for {len(streams_without_preview)} streams...")

        for stream_info in streams_without_preview:
            username = stream_info["username"]
            api_url_liveinfo = f"https://api.xcam.gay/user/{username}/liveInfo"
            print(f"🌐 Acessando API liveInfo para {username}: {api_url_liveinfo}")

            try:
                response_liveinfo = requests.get(api_url_liveinfo)
                response_liveinfo.raise_for_status()
                data_liveinfo = response_liveinfo.json()

                m3u8_url = data_liveinfo.get("cdnURL") or data_liveinfo.get("edgeURL")

                if m3u8_url:
                    print(f"✅ URL .m3u8 encontrada em liveInfo para {username}.")
                    streams_from_liveinfo.append({"username": username, "src": m3u8_url})
                else:
                    print(f"⚠️ Nenhuma URL .m3u8 válida encontrada em liveInfo para o usuário: {username}.")

                time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                print(f"❌ Erro na requisição HTTP para a API liveInfo de {username}: {e}")
            except json.JSONDecodeError:
                print(f"❌ Erro ao decodificar resposta JSON da API liveInfo de {username}. Resposta: {response_liveinfo.text}")
            except Exception as e:
                print(f"❌ Erro inesperado ao acessar API liveInfo de {username}: {e}")

    final_streams_list = streams_from_main + streams_from_liveinfo
    print(f"\nFinal list for page {page}: Found a total of {len(final_streams_list)} streams with valid URLs after fallback.")

    return final_streams_list

In [ ]:
# Etapa 2 – Gravar stream com ffmpeg (ajustado para nome de arquivo e local temporário)
def gravar_stream(username, m3u8_url):
    # Captura o timestamp no início para consistência no nome do arquivo
    start_time_dt = datetime.now()
    data_str = start_time_dt.strftime("%d-%m-%Y")
    horario_str = start_time_dt.strftime("%H-%M")

    temp_filename = f"{username}_{start_time_dt.strftime('%Y%m%d_%H%M%S')}_temp.mp4"
    filepath = os.path.join(TEMP_OUTPUT_FOLDER, temp_filename)

    print(f"\n🎬 Iniciando gravação de: {username} (URL: {m3u8_url}) em {filepath}")

    ffmpeg_cmd = ["ffmpeg", "-i", m3u8_url, "-t", str(RECORD_SECONDS), "-c", "copy", "-y", filepath]

    start_time_process = time.time()
    process = None

    try:
        # Execução com feedback de progresso
        process = subprocess.Popen(ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True)
        elapsed_seconds = 0
        last_log_minute = -1
        while True:
            line = process.stdout.readline()
            if not line and process.poll() is not None:
                break
            if "time=" in line:
                try:
                    match = re.search(r"time=(\d+):(\d+):(\d+)", line)
                    if match:
                        h, m, s = map(int, match.groups())
                        elapsed_seconds = h * 3600 + m * 60 + s
                        if elapsed_seconds // 60 != last_log_minute:
                            log_progress(username, elapsed_seconds, RECORD_SECONDS)
                            last_log_minute = elapsed_seconds // 60
                except Exception:
                    pass

        process.wait()
        end_time_process = time.time()
        elapsed_seconds = round(end_time_process - start_time_process)
        log_progress(username, elapsed_seconds, RECORD_SECONDS)

        if process.returncode != 0:
            print(f"❌ FFmpeg falhou para {username}. Código de saída: {process.returncode}")
            return {
                'username': username,
                'filename': temp_filename,
                'filepath': filepath,
                'upload_success': False,
                'abyss_response': "Gravação FFmpeg falhou"
            }
        else:
            print(f"✅ Gravação FFmpeg finalizada para: {temp_filename}. Duração aproximada: {elapsed_seconds}s")

            tempo_formatado = format_seconds(elapsed_seconds)
            final_filename = f"{username}-{data_str}_{horario_str}_{tempo_formatado}.mp4"
            final_filepath = os.path.join(TEMP_OUTPUT_FOLDER, final_filename)

            try:
                os.rename(filepath, final_filepath)
                print(f"✅ Arquivo renomeado para: {final_filename}")
                filepath_for_upload = final_filepath
                filename_for_upload = final_filename
            except Exception as e:
                print(f"❌ Erro ao renomear arquivo {temp_filename} para {final_filename}: {e}")
                filepath_for_upload = filepath
                filename_for_upload = temp_filename

            success, abyss_resp = upload_to_abyss_and_update_json(filepath_for_upload, username, elapsed_seconds)

            return {
                'username': username,
                'filename': filename_for_upload,
                'filepath': filepath_for_upload,
                'upload_success': success,
                'abyss_response': abyss_resp
            }

    except FileNotFoundError:
        print(f"❌ Erro: Comando 'ffmpeg' não encontrado. Certifique-se de que foi instalado corretamente.")
        return {
            'username': username,
            'filename': None,
            'filepath': None,
            'upload_success': False,
            'abyss_response': "Comando FFmpeg não encontrado"
        }
    except Exception as e:
        print(f"❌ Erro inesperado durante a execução do FFmpeg para {username}: {e}")
        return {
            'username': username,
            'filename': None,
            'filepath': None,
            'upload_success': False,
            'abyss_response': f"Erro inesperado na execução do FFmpeg: {e}"
        }
    finally:
        if 'filepath' in locals() and os.path.exists(filepath):
            try:
                os.remove(filepath)
                print(f"🗑️ Arquivo temporário local removido: {filepath}")
            except Exception as e:
                print(f"⚠️ Não foi possível remover o arquivo temporário local {filepath}: {e}")

        if 'final_filepath' in locals() and os.path.exists(final_filepath) and final_filepath != filepath:
            try:
                os.remove(final_filepath)
                print(f"🗑️ Arquivo local renomeado removido: {final_filepath}")
            except Exception as e:
                print(f"⚠️ Não foi possível remover o arquivo local renomeado {final_filepath}: {e}")

In [ ]:
# Etapa 3 – Upload para Abyss.to e gerenciamento do rec.json
def upload_to_abyss_and_update_json(filepath, username, duration_seconds):
    file_name = os.path.basename(filepath)
    file_type = 'video/mp4'
    print(f"⬆️ Tentando upload de: {file_name} para Abyss.to...")

    upload_success = False
    abyss_response = "Upload falhou - Sem resposta"
    uploaded_url = None
    video_id = None

    try:
        with open(filepath, 'rb') as f:
            files = { 'file': (file_name, f, file_type) }
            response = requests.post(ABYSS_UPLOAD_URL, files=files)

            print(f"   Status code da resposta do Abyss.to: {response.status_code}")

            try:
                resp_json = response.json()
                abyss_response = resp_json
                print(f"   Resposta JSON do Abyss.to: {json.dumps(resp_json, indent=2)}")

                if resp_json.get('status'):
                    upload_success = True
                    uploaded_url = resp_json.get('url') or resp_json.get('urlIframe')
                    video_id = resp_json.get('video')

                    if uploaded_url:
                        print(f"📤 Upload realizado com sucesso! URL: {uploaded_url}")
                    else:
                        print(f"✅ Upload realizado com sucesso, mas URL ('url' ou 'urlIframe') não retornada na resposta JSON.")

                else:
                    error_message = resp_json.get('message', 'Mensagem de erro não fornecida')
                    print(f"❌ Falha no upload para Abyss.to. Status 'status': false. Mensagem: {error_message}")

            except json.JSONDecodeError:
                abyss_response = response.text
                print(f"⚠️ Resposta do Abyss.to não é JSON. Status: {response.status_code}. Resposta: {response.text}")

            except Exception as e:
                abyss_response = f"Erro inesperado ao processar JSON: {e} | Resposta: {response.text}"
                print(f"⚠️ Erro inesperado ao processar resposta do Abyss.to: {e}. Resposta: {response.text}")

    except FileNotFoundError:
        abyss_response = "Arquivo não encontrado para upload"
        print(f"❌ Erro: Arquivo não encontrado para upload: {filepath}")
    except Exception as e:
        abyss_response = f"Erro geral no upload: {e}"
        print(f"❌ Erro geral no upload: {e}")

    # --- Gerenciamento do rec.json (APENAS se upload_success for True) ---
    if upload_success:
        try:
            user_folder = os.path.join(BASE_DRIVE_FOLDER, "user", username)
            os.makedirs(user_folder, exist_ok=True)

            json_filepath = os.path.join(user_folder, "rec.json")

            parts = file_name.replace(".mp4", "").split('-')
            if len(parts) >= 4:
                json_data = parts[1]
                json_horario = parts[2]
                json_tempo = parts[3]
            else:
                print(f"⚠️ Nome do arquivo ({file_name}) não está no formato esperado para extração de data/hora/tempo. Usando fallback.")
                now = datetime.now()
                json_data = now.strftime("%d-%m-%Y")
                json_horario = now.strftime("%H-%M")
                json_tempo = format_seconds(duration_seconds)

            new_video_entry = {
                "video": video_id if video_id else "ID_não_retornado",
                "title": file_name.replace(".mp4", ""),
                "file": file_name,
                "url": uploaded_url if uploaded_url else "URL_não_retornada",
                "data": json_data,
                "horario": json_horario,
                "tempo": json_tempo
            }

            rec_data = {"username": username, "records": 0, "videos": []}
            if os.path.exists(json_filepath):
                with open(json_filepath, 'r', encoding='utf-8') as f:
                    try:
                        loaded = json.load(f)
                        if isinstance(loaded, dict) and "videos" in loaded and isinstance(loaded.get("videos"), list):
                            rec_data = loaded
                        else:
                            print(f"⚠️ Estrutura do rec.json inválida, sobrescrevendo.")
                    except Exception as e:
                        print(f"⚠️ Erro ao carregar JSON existente: {e}, sobrescrevendo.")
            rec_data["records"] += 1
            rec_data["videos"].append(new_video_entry)
            with open(json_filepath, 'w', encoding='utf-8') as f:
                json.dump(rec_data, f, indent=2, ensure_ascii=False)
            print(f"✅ Arquivo rec.json para {username} atualizado/criado em {json_filepath}")
        except Exception as e:
            print(f"❌ Erro ao gerenciar rec.json para {username}: {e}")
            abyss_response = f"Upload sucesso, mas erro no JSON: {e}"

    return upload_success, abyss_response

In [ ]:
# Etapa 4 – Processamento paginado e execução paralela
def process_page(page=1, limit=5):
    print(f"\n📄 Iniciando processamento da página {page}\n")
    streams = get_broadcasts(limit=limit, page=page)

    if not streams:
        print(f"\n🚫 Nenhuma stream encontrada com URL válida na página {page}. Encerrando o processamento desta página.")
        return False

    jobs = []
    results = multiprocessing.Manager().list()

    def worker(username, m3u8_url, results):
        result = gravar_stream(username, m3u8_url)
        results.append(result)

    print(f"🚀 Iniciando gravação paralela para {len(streams)} streams...")
    for stream in streams:
        username = stream["username"]
        m3u8_url = stream["src"]
        p = multiprocessing.Process(target=worker, args=(username, m3u8_url, results))
        jobs.append(p)
        p.start()

    for job in jobs:
        job.join()

    print(f"\n🏁 Todas as gravações e uploads da página {page} foram concluídos.")

    return True if streams else False

def main():
    page = 1
    limit = 10  # Você pode ajustar o limite
    print("🤖 Iniciando o processo principal de busca e gravação de streams...")
    while True:
        ok = process_page(page=page, limit=limit)
        if not ok:
            print("\nProcesso principal encerrado por falta de streams encontradas com URLs válidas.")
            break
        page += 1
        if page > 10:
            page = 1
        print(f"\nAguardando 5 segundos antes de processar a página {page}...")
        time.sleep(5)

    print("\n✨ Processo principal concluído.")

if __name__ == '__main__':
    try:
        if 'google.colab' in str(get_ipython()):
            main()
        else:
            print("Script não está rodando no Google Colab. Execute main() manualmente se desejar.")
    except NameError:
        print("Não está rodando em um ambiente IPython/Colab. Execute main() manualmente se desejar.")